# Particle Localization

We'll compare the performance of a dense neural network and of a convolutional neural network for the classification of blood smears in the Malaria dataset.

## Load Data

We'll load a dataset including two videos of optically trapped particles.

In [ ]:
import os
import glob

if not os.path.exists("particle_dataset"):
    os.system("git clone -b cm https://github.com/DeepTrackAI/particle_dataset")

train_path = os.path.join("particle_dataset", "particle_dataset")
train_video_path = glob.glob(os.path.join(train_path,'*.avi'))

print(f"{len(train_video_path)} training videos")

In [ ]:
train_video_path[0]

In [ ]:
import cv2
import numpy as np

def load_video(path, frames_to_load=100, image_size=51):
    video = cv2.VideoCapture(path) 
    data = []
    for _ in range(frames_to_load):
        (_, frame) = video.read()
        frame = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX) 
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) / 255 
        frame = cv2.resize(frame, (image_size, image_size)) 
        data.append(frame) 
    return np.array(data)

In [ ]:
low_noise_data = load_video(train_video_path[1])
high_noise_data = load_video(train_video_path[0])

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(2, 6, figsize=(25, 7))
for i in range(6):
    im = ax[0, i].imshow(low_noise_data[i], vmin=0, vmax=1, cmap="gray")
    ax[0, i].text(0, 5, "Frame {}".format(i), color="white", fontsize=12)

    ax[0, i].axis("off") 
    ax[1, i].imshow(high_noise_data[i], vmin=0, vmax=1, cmap="gray") 
    ax[1, i].text(0, 5, "Frame {}".format(i), color="white", fontsize=12) 
    ax[1, i].axis("off")
plt.subplots_adjust(wspace=.1, hspace=.1) 
plt.colorbar(im, ax=ax.ravel().tolist())
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor

class ParticleCenter():
    def __init__(self, images):
        self.images = images
        self.positions = []
        self.i = 0
        self.fig, self.ax = plt.subplots(1, 1, figsize=(5, 5))
        self.fig.canvas.header_visible = False
        self.fig.canvas.footer_visible = False
    def start(self):
        self.im = self.ax.imshow(self.images[self.i], cmap="gray", vmin=0, vmax=1)
        self.text = self.ax.text(3, 5, "Frame " + str(self.i+1) + " of " + str(len(self.images)), color="white", fontsize=12) 
        self.ax.axis("off")
        self.cursor = Cursor(self.ax, useblit=True, color="red", linewidth=1)
        self.cid = self.fig.canvas.mpl_connect("button_press_event", self.onclick)
        self.next_image()
        plt.show()
    def next_image(self):
        im = self.images[self.i]
        self.im.set_data(im)
        self.text.set_text("Frame " + str(self.i+1) + " of " + str(len(self.images)))
        self.fig.canvas.draw_idle()
    def onclick(self, event):
        self.positions.append([event.xdata, event.ydata])
        if self.i < len(self.images)-1:
            self.i += 1
            self.next_image()
        else:
            self.fig.canvas.mpl_disconnect(self.cid)
            plt.close()
            return

In [ ]:
%matplotlib notebook
from numpy.random import choice

number_of_images_to_annotate = 50

dataset = np.concatenate([low_noise_data, high_noise_data], axis=0)
np.random.shuffle(dataset)

images_to_annotate = choice(
  np.arange(dataset.shape[0]),
  number_of_images_to_annotate,
  replace=False
)

pc = ParticleCenter(dataset[images_to_annotate])
pc.start()

annotated_data = pc.images 
labels = pc.positions


In [ ]:
split_ratio = 0.8
train_data = annotated_data[:int(split_ratio * len(annotated_data))]
train_labels = labels[:int(split_ratio * len(labels))]
np.save(os.path.join(train_path, "train_data.npy"), np.array(train_data))
np.save(os.path.join(train_path, "train_labels.npy"), np.array(train_labels))

In [ ]:
remaining_images = np.delete(dataset, images_to_annotate, axis=0)
np.save(os.path.join(train_path, "test_data.npy"), np.array(remaining_images))

In [ ]:
import deeplay as dl

CNN = dl.Sequential(
    dl.ConvolutionalNeuralNetwork(in_channels = 1, hidden_channels = [16, 32, 64], out_channels = 64),
    #dl.MultiLayerPerceptron(in_features = 64, hidden_features = [], out_features = 1, out_activation = torch.nn.Sigmoid)
)
#CNN[0].blocks[2].pool.configure(torch.nn.MaxPool2d, kernel_size = 2)

In [ ]:
print(CNN)

We'll define the path to the directories containing the `Infected` and `Parasitized` images.

## Visualize Data

We'll then visualize some of the data.

In [ ]:
from blood_smears import plot_examples
plot_examples(uninfected_files,infected_files)

## Data Preprocessing

We'll define a popeline to resize and normalize the data ...

In [ ]:
import torchvision.transforms as tt

pipeline = tt.Compose([tt.Resize((28,28)),
    tt.ToTensor()]) # automatically converts images to [0,1]

... subset the full dataset and split it into `train` and `test` sets ...

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import Lambda
from torch.utils.data import random_split, Subset

dataset = ImageFolder(base_dir, pipeline, target_transform= Lambda(lambda y: torch.tensor(abs(1-y)).float().unsqueeze(-1)))

subset_size = 5000
subset_indices = torch.randperm(len(dataset))[:subset_size]
subset = Subset(dataset, subset_indices)

train_size = int(0.8 * len(subset))
test_size = len(subset) - train_size
train, test = random_split(subset, [train_size, test_size])

... and define the dataloaders for both sets. For the training, we'll set `batch_size = 32`.

In [ ]:
from torch.utils.data import DataLoader
batch_size = 32
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size * 5, shuffle=False)

## Fully-connected Neural Network

We'll define a Fully-connected Neural Network (FCNN) using `deeplay`. The FCNN has 2 layers with 128 features.

In [ ]:
import deeplay as dl

FCNN = dl.MultiLayerPerceptron(in_features = 28 * 28 * 3,
                                hidden_features = [128, 128],
                                out_features = 1,
                                out_activation = torch.nn.Sigmoid,
)
FCNN.blocks.activation.configure(torch.nn.Sigmoid)

We'll define a classifier based on the FCNN architecture, including loss function, evaluation metrics and othe hyperparameters ...

In [ ]:
import torchmetrics as tm

FCNN_classifier_template = dl.BinaryClassifier(
        model=FCNN,
        optimizer=dl.RMSprop(lr=.001),
        )

FCNN_classifier = FCNN_classifier_template.create()
print(FCNN_classifier)

... and a trainer including other hyperparameters ...

In [ ]:
FCNN_trainer = dl.Trainer(
    max_epochs=20, # How many times to run through the entire dataset
    accelerator="auto", # Use GPU if available
)

We'll start the training and visualize the evaluation metrics.

In [ ]:
FCNN_trainer.fit(FCNN_classifier, train_loader)

We'll calculate the performance over the test set.

In [ ]:
_ = FCNN_trainer.test(FCNN_classifier, test_loader)

In [ ]:
from blood_smears import plot_ROC_AUC
_,_,_,_ = plot_ROC_AUC(classifier = FCNN_classifier, dataset=test)

## Convolutional Neural Network with a dense top
We'll now build a convolutional neural network (CNN) with a FCNN at the end ...


... define a classifier using the CNN ...

In [ ]:
CNN_classifier_template = dl.BinaryClassifier(
    model=CNN, 
    optimizer=dl.RMSprop(lr=.001),
)

CNN_classifier = CNN_classifier_template.create()
print(CNN_classifier)

... train it ...

In [ ]:
CNN_trainer = dl.Trainer(
    max_epochs=20, # How many times to run through the entire dataset
    accelerator="auto", # Use GPU if available
)

CNN_trainer.fit(CNN_classifier, train_loader)

... evaluate the performance over the test set ...

In [ ]:
_ = CNN_trainer.test(CNN_classifier, test_loader)

... and display the ROC curve with the AUC value.

In [ ]:
images, gt, pred, _ = plot_ROC_AUC(classifier=CNN_classifier, dataset=test)

## Failure analysis
We'll use a function to visualize some of the wrongly classified cells, looking for common patterns.
```python
def plot_failure(images, gt, pred, threshold = 0.5, num_of_plots = 5):
    from matplotlib import pyplot as plt 
    from numpy import array, squeeze   
    
    pred = array(pred).squeeze()
    gt = array(gt).squeeze()
    images = array(images)

    pred_class = pred > threshold

    false_positives = (pred_class == 1) & (gt == 0)
    false_positives_images = images[false_positives]

    false_negatives = (pred_class == 0) & (gt == 1)
    false_negatives_images = images[false_negatives]

    plt.figure(figsize=(num_of_plots*2, 5))
    for i in range(num_of_plots):

        # false positives
        plt.subplot(2, num_of_plots, i + 1)
        plt.imshow(false_positives_images[i].transpose(1, 2, 0))
        if i == 0:
            plt.title("False positives", fontsize=16, y=1.1)

        # false negatives
        plt.subplot(2, num_of_plots, i + num_of_plots + 1)
        plt.imshow(false_negatives_images[i].transpose(1, 2, 0))
        if i == 0:
            plt.title("False negatives", fontsize=16, y=1.1)

    plt.tight_layout()
    plt.show()
```

In [ ]:
from blood_smears import plot_failure
plot_failure(images=images, gt=gt, pred=pred, threshold = 0.5, num_of_plots = 4)

## Filters
We can access and visualize the filters used by the network at a specific layer.

In [ ]:
weights = CNN_classifier.model[0].input_block.layer.weight
w = weights.clone().detach()

from blood_smears import plot_filters_activations
plot_filters_activations(input = w, n_rows=4, label = 'Filters', normalize = True)

## Activations and Grad-CAM
To visualize the network feautures, we'll use `hooks`, functions that allows us to access the information that the model sees during forward and backward passes, such as activations and gradients, respectively. We'll define them as context manager classes, so that we can use them with the `with` statement:
```python
class fwd_hook():
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_func)   
    def hook_func(self, m, i, o):
        print('Forward hook running...') 
        self.stored = o.detach().clone()
        print(f'Activations size: {self.stored.size()}')
    def __enter__(self, *args): 
        return self
    def __exit__(self, *args): 
        self.hook.remove()

class bwd_hook():
    def __init__(self, m):
        self.hook = m.register_full_backward_hook(self.hook_func)
    def hook_func(self, m, gi, go):
        print('Backward hook running...')
        self.stored = go[0].detach().clone()
        print(f'Gradients size: {self.stored.size()}')
    def __enter__(self, *args): 
        return self
    def __exit__(self, *args): 
        self.hook.remove()
```

We'll randomly pick the image of an infected smear.

In [ ]:
import numpy as np
import matplotlib.image as mpimg

ind_infect = np.where((torch.cat(gt)==1).tolist())[0]
ind=np.random.choice(ind_infect,1)[0]

test_image = images[ind]
test_image_hr=mpimg.imread(dataset.imgs[subset.indices[test.indices[ind]]][0])

Activations and gradients at a specific layer can be obtained from the forward and backward pass, respectively ...

In [ ]:
from blood_smears import fwd_hook, bwd_hook

test_layer = CNN_classifier.model[0].blocks[3].layer

with bwd_hook(test_layer) as bh:
    with fwd_hook(test_layer) as fh:
        out = CNN_classifier.model(test_image.unsqueeze(0)).backward()
activations = fh.stored
gradients = bh.stored

... we can plot the activations ...

In [ ]:
plot_filters_activations(input = activations.permute(1,0,2,3),n_rows=8,label = 'Feature maps', normalize = False)

... or combine gradients and activations to calculate Grad-CAM and inspect on which part of an image the CNN focuses on to predict its outputs. 

In [ ]:
pooled_grad = gradients[0].mean(dim=[1,2], keepdim = True)
grad_cam = torch.nn.functional.relu((pooled_grad*activations[0]).sum(0)).detach().numpy()

from blood_smears import plot_gradcam
plot_gradcam(grad_cam, test_image_hr)